Adapted from : https://www.kaggle.com/code/theoviel/dicom-resized-png-jpg

**Dataset Links :**
- Part 1 : https://www.kaggle.com/datasets/theoviel/rsna-abdominal-trauma-detection-png-pt1
- Part 2 : https://www.kaggle.com/datasets/theoviel/rsna-abdominal-trauma-detection-png-pt2
- Part 3 : https://www.kaggle.com/datasets/theoviel/rsna-2023-abdominal-trauma-detection-pngs-3-8
- Part 4 : https://www.kaggle.com/datasets/theoviel/rsna-abdominal-trauma-detection-png-pt4
- Part 5 : https://www.kaggle.com/datasets/theoviel/rsna-abdominal-trauma-detection-png-pt5
- Part 6 : https://www.kaggle.com/datasets/theoviel/rsna-abdominal-trauma-detection-png-pt6
- Part 7 : https://www.kaggle.com/datasets/theoviel/rsna-abdominal-trauma-detection-pngs-pt7
- Part 8 : https://www.kaggle.com/datasets/theoviel/rsna-2023-abdominal-trauma-detection-pngs-18

**Changes :**
- Apply `standardize_pixel_array` function
- Update links

**TODO :**
- Dicom processing on GPU
- Figure out why example dicom is too dark

In [ ]:
#!pip install -qU python-gdcm pydicom pylibjpeg
#!pip install opencv-python
#!pip install seaborn

In [ ]:
import os
import cv2
import glob
import gc
import gdcm
import pydicom
import zipfile
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from joblib import Parallel, delayed
from pydicom.pixel_data_handlers.util import apply_voi_lut
from multiprocessing import Pool

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
def standardize_pixel_array(dcm: pydicom.dataset.FileDataset) -> np.ndarray:
    """
    Source : https://www.kaggle.com/competitions/rsna-2023-abdominal-trauma-detection/discussion/427217
    """
    # Correct DICOM pixel_array if PixelRepresentation == 1.
    pixel_array = dcm.pixel_array
    if dcm.PixelRepresentation == 1:
        bit_shift = dcm.BitsAllocated - dcm.BitsStored
        dtype = pixel_array.dtype 
        new_array = (pixel_array << bit_shift).astype(dtype) >>  bit_shift
        pixel_array = pydicom.pixel_data_handlers.util.apply_modality_lut(new_array, dcm)
    return pixel_array

In [ ]:
TRAIN_PATH = "/home/junseonglee/01_codes/input/rsna-2023-abdominal-trauma-detection/train_images/"
BASE_PATH = '/home/junseonglee/01_codes/input/rsna-2023-abdominal-trauma-detection'

print('Number of training patients :', len(os.listdir(TRAIN_PATH)))

In [ ]:
counter = 0
for patient in sorted(os.listdir(TRAIN_PATH)):
    for study in os.listdir(TRAIN_PATH + patient):
        counter+=1
        if(counter < 15):
            continue
        if(counter > 16):
            break
        print(patient)
        print(study)
        imgs = {}
        for f in sorted(glob.glob(TRAIN_PATH + f"{patient}/{study}/*.dcm"))[::10]:
            dicom = pydicom.dcmread(f)

            pos_z = dicom[(0x20, 0x32)].value[-1]  # to retrieve the order of frames

            img = standardize_pixel_array(dicom)
            
            #img = (img - img.min()) / (img.max() - img.min() + 1e-6)
            #junseonglee11 it seems there are some outliers that make img max very big --> quantile?
            quant_0_95 = np.quantile(img, 0.8)
            quant_0_05 = np.quantile(img, 0.2)
            img = (img - quant_0_05) / (quant_0_95 - quant_0_05 + 1e-6)
            
            if dicom.PhotometricInterpretation == "MONOCHROME1":
                img = 1 - img

            imgs[pos_z] = img
        print(np.shape(img))
        for i, k in enumerate(sorted(imgs.keys())):
            img = imgs[k]
            
            if not (i % 100):
                plt.figure(figsize=(5, 5))
                plt.imshow(img, cmap="gray")
                plt.title(f"Patient {patient} - Study {study} - Frame {i}/{len(imgs)}")
                plt.axis(False)
                plt.show()
    
        #break
    #break

### Save the processed data

In [ ]:
def process(patient, size=512, save_folder="", data_path=""):
    for study in sorted(os.listdir(data_path + patient)):
        imgs = {}
        for f in sorted(glob.glob(data_path + f"{patient}/{study}/*.dcm")):
            dicom = pydicom.dcmread(f)

            pos_z = dicom[(0x20, 0x32)].value[-1]

            img = standardize_pixel_array(dicom)
            img = (img - img.min()) / (img.max() - img.min() + 1e-6)

            if dicom.PhotometricInterpretation == "MONOCHROME1":
                img = 1 - img

            imgs[pos_z] = img

        for i, k in enumerate(sorted(imgs.keys())):
            img = imgs[k]
            if size is not None:
                img = cv2.resize(img, (size, size))

            if isinstance(save_folder, str):
                cv2.imwrite(save_folder + f"{patient}_{study}_{i}.png", (img * 255).astype(np.uint8))
            else:
                im = cv2.imencode('.png', (img * 255).astype(np.uint8))[1]
                save_folder.writestr(f'{patient}_{study}_{i:04d}.png', im)

In [ ]:
n_chunk = 8
patients = os.listdir(TRAIN_PATH)
n_patients = len(patients)
rng_patients = np.linspace(0, n_patients+1, n_chunk+1, dtype = int)
rng_patients
patients

In [ ]:
data_path = TRAIN_PATH
save_folder = BASE_PATH + '/3d_preprocessed'
RESOL = 256

for i in tqdm(range(0, len(patients))):
    patient = patients[i]
    for study in sorted(os.listdir(data_path + patient)):
        imgs = {}
        
        for f in sorted(glob.glob(data_path + f"{patient}/{study}/*.dcm")):
            dicom = pydicom.dcmread(f)

            pos_z = dicom[(0x20, 0x32)].value[-1]

            img = standardize_pixel_array(dicom)
            #img = (img - img.min()) / (img.max() - img.min() + 1e-6)
            imgs[pos_z] = img
        #print(len(imgs))
        sample_z = np.linspace(0, len(imgs)-1, 256, dtype=int)
        #sample_z = np.round(sample_z).astype(int)
        imgs_3d = []
        for i, k in enumerate(sorted(imgs.keys())):
            if i in sample_z:
                img = imgs[k]
                imgs_3d.append(cv2.resize(img, (RESOL, RESOL))[None])
        imgs_3d = np.vstack(imgs_3d)
        nu = np.zeros((RESOL, RESOL, RESOL))
        for i in range(0, len(imgs_3d[0,0])):
            nu[:,:,i] = cv2.resize(imgs_3d[:,:,i], (RESOL, RESOL))
        imgs_3d  = nu
        imgs_3d = ((imgs_3d - imgs_3d.min()) / (imgs_3d.max() - imgs_3d.min()))

        if dicom.PhotometricInterpretation == "MONOCHROME1":
            imgs_3d = 1 - imgs_3d
        imgs_3d*=255
        imgs_3d = imgs_3d.astype(np.uint8)
        
        imgs_3d = imgs_3d.reshape(256, RESOL*RESOL)
        cv2.imwrite(f'{save_folder}/{patient}_{study}.png', imgs_3d)

        del imgs_3d, imgs, img, nu
        gc.collect()
            
   

In [36]:
len(glob.glob(f'{save_folder}/*'))

4711

In [ ]:
train_df = pd.read_csv(f'{BASE_PATH}/train.csv')
train_meta = pd.read_csv(f'{BASE_PATH}/train_series_meta.csv')
train_df = train_df.sort_values(by=['patient_id'])
train_df

TRAIN_PATH = BASE_PATH + "/train_images/"
n_chunk = 8
patients = os.listdir(TRAIN_PATH)
n_patients = len(patients)
rng_patients = np.linspace(0, n_patients+1, n_chunk+1, dtype = int)
patients_cts = glob.glob(f'{TRAIN_PATH}/*/*')
n_cts = len(patients_cts)
patients_cts_arr = np.zeros((n_cts, 2), int)
data_paths=[]
for i in range(0, n_cts):
    patient, ct = patients_cts[i].split('/')[-2:]
    patients_cts_arr[i] = patient, ct
    data_paths.append(f'{BASE_PATH}/3d_preprocessed/{patients_cts_arr[i,0]}_{patients_cts_arr[i,1]}.png')
TRAIN_IMG_PATH = BASE_PATH + '/processed' 

#Generate tables for training
train_meta_df = pd.DataFrame(patients_cts_arr, columns = ['patient_id', 'series'])

#5-fold splitting
train_df['fold'] = 0
labels = train_df[['bowel_healthy','bowel_injury',
                    'extravasation_healthy','extravasation_injury',
                    'kidney_healthy','kidney_low','kidney_high',
                    'liver_healthy','liver_low','liver_high',
                    'spleen_healthy','spleen_low','spleen_high',
                    'any_injury']].to_numpy()

mskf = MultilabelStratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=0)
counter = 0
for train_index, test_index in mskf.split(np.ones(len(train_df)), labels):
    for i in range(0, len(test_index)):
        train_df['fold'][test_index[i]] = counter
    counter+=1

train_meta_df = train_meta_df.join(train_df.set_index('patient_id'), on='patient_id')
train_meta_df['path']=data_paths
train_meta_df.to_csv(f'{BASE_PATH}/train_meta.csv', index = False)
np.unique(train_df['fold'].to_numpy(), return_counts = True)


Done ! 